# Ni Super Alloy Design Workflow



In [1]:
import numpy as np
from xml_io import load_variables
from xml_io import save_variables
import time

## Precipiations Kinetics Model

### Load input variables

In [2]:
# def load_input_parameters():
import xmltodict

GA_select = []
with open('xmls/GA_select.xml') as inpread:
	GA_select = xmltodict.parse(inpread.read())

gen = int(GA_select['ngcInputs']['noGeneration'])
n_samples = int(GA_select['ngcInputs']['noSamples'])
start_samples = int(GA_select['ngcInputs']['startSamples'])

n_variables = len(GA_select['ngcInputs']['variables']['value'])
variables = GA_select['ngcInputs']['variables']['value']

print gen
print n_samples
print start_samples

print n_variables
print variables

# variables = []

# GA_inputs = []
# with open('GA_inputs.xml') as inpread:
#         GA_inputs = xmltodict.parse(inpread.read())

# input_xml_ID = str(GA_inputs['gaInputs']['iD'])
# Search_domain = [['' for x in xrange(2)] for j in range(no_variables)]
# for i in range(0,no_variables):
# 	Search_domain[i][0] = GA_inputs['gaInputs']['searchDomain']['variable'][i]  # number after ['item']
# 	Search_domain[i][1] = GA_inputs['gaInputs']['searchDomain']['unit'][i]


# for i in range (no_variables):
# 	variables.append(float(GA_select['ngcInputs']['variables']['value'][i]))

# no_variables = no_variables/no_samples

# T_service = float(GA_select['ngcInputs']['serviceTemperature']['value'])

#thermo_input_xml_file_path = 'xmls/GA_select.xml'
#thermo_input_file_key = 'ngcInputs'
#thermo_value_keys = ['noGeneration', 'noSamples', 'startSamples']
#gen, n_samples, start_samples = load_variables(thermo_input_xml_file_path,
#                                                            thermo_input_file_key,
#                                                            thermo_value_keys)  


0
1
0
3
[u'0.13', u'0.12', u'1030']


### Find optimal volume fraction and average radius

In [4]:
%run NGC_NIST.py



[0.13, 0.12, 1030.0] 1123.0
volume fraction: 0.78135101015
mean radius: 4.36388632273e-08
yield stress: 840.269416854
precipitation stress: 654.839831193
processing time: 66211.4401737


### Save output variabes

## Elastic Deformation Model

### Load Input Variables

In [ ]:
pymks_input_xml_file_path = 'xmls/Kinetics.xml'
pymks_input_file_key = 'kinetics'
pymks_value_keys = ['volumeFraction', 'averageRadius', 'serviceTemperature']
vf, ra, T_service = load_variables(pymks_input_xml_file_path,
                                   pymks_input_file_key, pymks_value_keys)

### Find Optimal Radius

In [ ]:
from pymks_elasticity import pymks_elasticity

Elasticity = np.array([0], dtype=float)

print "PyMKS:", time.asctime(time.localtime(time.time()))
Elasticity = pymks_elasticity(vf, ra, T_service, 1)
print Elasticity

### Save Output Variables

In [ ]:
output_MKS= []
output_MKS.append(('optimumVolumeFraction', vf, '%'))
output_MKS.append(('optimumRadius', ra, 'meter'))
output_MKS.append(('youngsModulus', Elasticity, 'MPa'))

save_variables('PyMKS.xml', 'pyMks', output_MKS)

## Plastic Deformation Model

### Load Input Variables

In [ ]:
plasticty_input_xml_file_path = 'xmls/Kinetics.xml'
plasticty_input_file_key = 'kinetics'
plasticty_value_keys = ['volumeFraction', 'averageRadius', 'serviceTemperature',
                    'precipitateStress', 'solidSolutionStress']
vf, ra, T_service, prec_stress, SS_stress = load_variables(plasticty_input_xml_file_path,
                                                           plasticty_input_file_key,
                                                           plasticty_value_keys)
youngs_modulus = load_variables('xmls/PyMKS.xml', 'pyMks', ['youngsModulus'])


### Compute Stress-Strain Curve and $E_{WTN}$

In [ ]:
from plastic_deformation import stress_strain

stress_strain, work_to_necking = stress_strain(youngs_modulus, vf, prec_stress,
                                               SS_stress, T_service)

print 'UTS, MPa: ', stress_strain[-1, 0], '  Strain_UTS, %:', \
      stress_strain[-1, 1], '   Work to Necking:', work_to_necking

### Save output variables